In [34]:
import numpy as np

# Definir las tasas de llegada y servicio según los parámetros
tasa_llegada_retiro = {
    'rápido': 1/1,
    'normal': 1/2,
    'lento': 1/3,
    'muy lento': 1/3
}

tasa_servicio_retiro = {
    'rápido': 1/1,
    'normal': 1/2,
    'lento': 1/3,
    'muy lento': 1/4
}

tasa_llegada_pago = {
    'rápido': 1/1,
    'normal': 1/2,
    'lento': 1/3,
    'muy lento': 1/4
}

tasa_servicio_pago = {
    'rápido': 1/3,
    'normal': 1/3,
    'lento': 1/5,
    'muy lento': 1/7
}

# Probabilidades de tipos de usuario
prob_retiro = [0.23, 0.40, 0.17, 0.20]
prob_pago = [0.10, 0.20, 0.30, 0.40]

# Tipos de usuario
tipos_de_usuario = ['rápido', 'normal', 'lento', 'muy lento']



In [35]:
# Simular un día de operaciones en el banco
def simulate_day():
    total_usarios = 0
    tiempos_espera = []
    contar_usuarios = {ut: 0 for ut in tipos_de_usuario}
    tiempos_por_caja = [0, 0, 0]  # Para rastrear el tiempo total de servicio por cada caja
    
    for cajero in range(3):  # Tres cajeros
        tiempo = 0
        queue = []
        while tiempo < 8 * 60:  # 8 horas de operación
            # Decidir si es retiro o pago
            if np.random.rand() >= 0.3:
                tipo_accion = 'retiro'
                tasa_llegada = np.random.choice(list(tasa_llegada_retiro.values()), p=prob_retiro)
                tasa_servicio = np.random.choice(list(tasa_servicio_retiro.values()), p=prob_retiro)
            else:
                tipo_accion = 'pago'
                tasa_llegada = np.random.choice(list(tasa_llegada_pago.values()), p=prob_pago)
                tasa_servicio = np.random.choice(list(tasa_servicio_pago.values()), p=prob_pago)

            # Tiempo hasta la próxima llegada
            interarrival_time = np.random.exponential(scale=1/tasa_llegada)
            tiempo += interarrival_time
            if tiempo >= 8 * 60:
                break
            
            # Tipo de usuario
            tipo_usr = np.random.choice(tipos_de_usuario, p=prob_retiro if tipo_accion == 'retiro' else prob_pago)
            contar_usuarios[tipo_usr] += 1
            
            # Tiempo de servicio
            tiempo_servicio = np.random.exponential(scale=1/tasa_servicio)
            if queue:
                queue[-1][1] += tiempo_servicio  # Agregar el tiempo de servicio al último usuario en la cola
            queue.append([tiempo, tiempo_servicio])
            total_usarios += 1
        
        # Calcular tiempos de espera y tiempo total de servicio para cada caja
        tiempo_inicio_servicio = 0
        for tiempo_llegada, tiempo_servicio in queue:
            if tiempo_inicio_servicio < tiempo_llegada:
                tiempo_inicio_servicio = tiempo_llegada
            tiempo_de_espera = tiempo_inicio_servicio - tiempo_llegada + tiempo_servicio
            tiempos_espera.append(tiempo_de_espera)
            tiempo_inicio_servicio += tiempo_servicio
            tiempos_por_caja[cajero] += tiempo_servicio  # Sumar el tiempo de servicio al tiempo total de la caja
    
        print(
            f"Tipo de accion:" , tipo_accion.swapcase(), 
            "\n\tTiempo de Servicio:" , tiempo_servicio , 
            "\n\t\t Usuario Atendido en caja" , cajero + 1 , "\n"            
            )

    promedio_tiempo_espera = np.mean(tiempos_espera)
    return promedio_tiempo_espera, contar_usuarios, total_usarios, tiempos_por_caja




In [36]:
# Realizar múltiples corridas
def simulacion_banco(num_simulaciones):
    results = []
    for _ in range(num_simulaciones):
        results.append(simulate_day())
    return results



In [37]:
# Ejecutar la simulación
simulaciones = 10
result = simulacion_banco(simulaciones)

# Calcular estadísticas
promedio_esperas = [r[0] for r in result]
conteo_usuarios_general = [r[1] for r in result]
total_usuarios_general = [r[2] for r in result]
tiempos_por_caja_general = [r[3] for r in result]

promedio_espera_general = np.mean(promedio_esperas)
promedio_usuarios = {ut: np.mean([user_counts[ut] for user_counts in conteo_usuarios_general]) for ut in tipos_de_usuario}

# Calcular la caja con el menor y mayor tiempo de servicio
tiempos_por_caja_promedio = np.mean(tiempos_por_caja_general, axis=0)
caja_menor_tiempo = np.argmin(tiempos_por_caja_promedio) + 1
caja_mayor_tiempo = np.argmax(tiempos_por_caja_promedio) + 1



Tipo de accion: RETIRO 
	Tiempo de Servicio: 0.9803294071179348 
		 Usuario Atendido en caja 1 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 12.741039024427835 
		 Usuario Atendido en caja 2 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 1.4227811082645578 
		 Usuario Atendido en caja 3 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 0.7210035658855457 
		 Usuario Atendido en caja 1 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 0.13345603083654434 
		 Usuario Atendido en caja 2 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 0.44173529559094027 
		 Usuario Atendido en caja 3 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 3.4816247252609123 
		 Usuario Atendido en caja 1 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 10.34093911473306 
		 Usuario Atendido en caja 2 

Tipo de accion: PAGO 
	Tiempo de Servicio: 4.022601483744015 
		 Usuario Atendido en caja 3 

Tipo de accion: RETIRO 
	Tiempo de Servicio: 0.42609169109844974 
		 Usuario Atendido en caja 1 

Tipo de accion: RETIRO 
	Tiempo

In [38]:
print("\nTiempo promedio de espera:", promedio_espera_general , "\n")
print("Promedio de usuarios de cada tipo:", promedio_usuarios , "\n")
print("Total de usuarios en todas las corridas:", total_usuarios_general , "\n")


Tiempo promedio de espera: 411.2003756130786 

Promedio de usuarios de cada tipo: {'rápido': 114.1, 'normal': 208.6, 'lento': 126.7, 'muy lento': 161.8} 

Total de usuarios en todas las corridas: [625, 586, 654, 579, 612, 580, 630, 642, 590, 614] 



In [39]:
for idx, usuarios in enumerate(total_usuarios_general, start=1):
    print(f"El Total de usuarios en corrida" , idx , "es:" , usuarios)



El Total de usuarios en corrida 1 es: 625
El Total de usuarios en corrida 2 es: 586
El Total de usuarios en corrida 3 es: 654
El Total de usuarios en corrida 4 es: 579
El Total de usuarios en corrida 5 es: 612
El Total de usuarios en corrida 6 es: 580
El Total de usuarios en corrida 7 es: 630
El Total de usuarios en corrida 8 es: 642
El Total de usuarios en corrida 9 es: 590
El Total de usuarios en corrida 10 es: 614


In [40]:
print("\nTiempos de servicio promedio por caja:", tiempos_por_caja_promedio , "\n")
print(f"La caja con menor tiempo de servicio es la caja {caja_menor_tiempo} con un tiempo de servicio promedio de {tiempos_por_caja_promedio[caja_menor_tiempo-1]}")
print(f"La caja con mayor tiempo de servicio es la caja {caja_mayor_tiempo} con un tiempo de servicio promedio de {tiempos_por_caja_promedio[caja_mayor_tiempo-1]}")


Tiempos de servicio promedio por caja: [1243.217951   1291.92279998 1302.69831988] 

La caja con menor tiempo de servicio es la caja 1 con un tiempo de servicio promedio de 1243.2179509964387
La caja con mayor tiempo de servicio es la caja 3 con un tiempo de servicio promedio de 1302.6983198796756


In [41]:
promedio_usuarios_total = {ut: np.mean([user_counts[ut] for user_counts in conteo_usuarios_general]) for ut in tipos_de_usuario}

print("Promedio de usuarios de cada tipo en la totalidad de cajeros:")
for tipo, promedio in promedio_usuarios_total.items():
    print(f"{tipo}: {promedio}")


Promedio de usuarios de cada tipo en la totalidad de cajeros:
rápido: 114.1
normal: 208.6
lento: 126.7
muy lento: 161.8


In [42]:
# Obtener el total de usuarios de cada tipo en cada simulación
usuarios_por_tipo_por_simulacion = [{ut: user_counts[ut] for ut in tipos_de_usuario} for user_counts in conteo_usuarios_general]

# Encontrar la simulación con el menor número de usuarios por tipo
simulacion_menor_usuarios_por_tipo = min(usuarios_por_tipo_por_simulacion, key=lambda x: sum(x.values()))

print("\nTotal de usuarios de cada tipo en cada simulación:")
for idx, usuarios_por_tipo in enumerate(usuarios_por_tipo_por_simulacion, start=1):
    print(f"Simulación {idx}: {usuarios_por_tipo}")

print("\nSimulación con menor cantidad de usuarios por tipo:")
print(simulacion_menor_usuarios_por_tipo)



Total de usuarios de cada tipo en cada simulación:
Simulación 1: {'rápido': 131, 'normal': 217, 'lento': 126, 'muy lento': 151}
Simulación 2: {'rápido': 95, 'normal': 201, 'lento': 123, 'muy lento': 167}
Simulación 3: {'rápido': 125, 'normal': 224, 'lento': 127, 'muy lento': 178}
Simulación 4: {'rápido': 104, 'normal': 195, 'lento': 122, 'muy lento': 158}
Simulación 5: {'rápido': 127, 'normal': 207, 'lento': 131, 'muy lento': 147}
Simulación 6: {'rápido': 107, 'normal': 212, 'lento': 119, 'muy lento': 142}
Simulación 7: {'rápido': 114, 'normal': 206, 'lento': 130, 'muy lento': 180}
Simulación 8: {'rápido': 114, 'normal': 230, 'lento': 133, 'muy lento': 165}
Simulación 9: {'rápido': 110, 'normal': 184, 'lento': 123, 'muy lento': 173}
Simulación 10: {'rápido': 114, 'normal': 210, 'lento': 133, 'muy lento': 157}

Simulación con menor cantidad de usuarios por tipo:
{'rápido': 104, 'normal': 195, 'lento': 122, 'muy lento': 158}


In [43]:
umbral_tiempo_espera = 10  # Umbral en minutos

if promedio_espera_general > umbral_tiempo_espera:
    print(f"Es necesario crear un nuevo cajero debido al alto tiempo promedio de espera ({promedio_espera_general} minutos).")
else:
    print(f"No es necesario crear un nuevo cajero, el tiempo promedio de espera es aceptable ({promedio_espera_general} minutos).")


Es necesario crear un nuevo cajero debido al alto tiempo promedio de espera (411.2003756130786 minutos).


In [45]:
# Calcular tiempos promedio de servicio por tipo de operación
tiempos_promedio_servicio_retiro = np.mean(tiempos_por_caja_promedio)  # Primeros 3 elementos para retiros
tiempos_promedio_servicio_pago = np.mean(tiempos_por_caja_promedio)   # Últimos 3 elementos para pagos

# Decidir cuántos cajeros deben ofrecer atención exclusiva para pagos y retiros
proporcion_retiro = tiempos_promedio_servicio_retiro / (tiempos_promedio_servicio_retiro + tiempos_promedio_servicio_pago)
proporcion_pago = tiempos_promedio_servicio_pago / (tiempos_promedio_servicio_retiro + tiempos_promedio_servicio_pago)

print(f"\nProporción de cajeros para retiros: {proporcion_retiro:.2f}")
print(f"Proporción de cajeros para pagos: {proporcion_pago:.2f}")



Proporción de cajeros para retiros: 0.50
Proporción de cajeros para pagos: 0.50
